# Publications markdown generator for academicpages

Takes a TSV of publications with metadata and converts them for use with [academicpages.github.io](academicpages.github.io). This is an interactive Jupyter notebook ([see more info here](http://jupyter-notebook-beginner-guide.readthedocs.io/en/latest/what_is_jupyter.html)). The core python code is also in `publications.py`. Run either from the `markdown_generator` folder after replacing `publications.tsv` with one containing your data.

TODO: Make this work with BibTex and other databases of citations, rather than Stuart's non-standard TSV format and citation style.


## Data format

The TSV needs to have the following columns: pub_date, title, venue, excerpt, citation, site_url, and paper_url, with a header at the top. 

- `excerpt` and `paper_url` can be blank, but the others must have values. 
- `pub_date` must be formatted as YYYY-MM-DD.
- `url_slug` will be the descriptive part of the .md file and the permalink URL for the page about the paper. The .md file will be `YYYY-MM-DD-[url_slug].md` and the permalink will be `https://[yourdomain]/publications/YYYY-MM-DD-[url_slug]`

This is how the raw file looks (it doesn't look pretty, use a spreadsheet or other program to edit and create).

In [15]:
!cat publications.tsv

pub_date	title	venue	excerpt	citation	url_slug	paper_url
05/06/2023	Poly-GAN: Regularizing Polygons with Generative Adversarial Networks	Lecture Notes in Computer Science	"This paper introduces Poly-GAN, a data-driven approach utilizing Deep Learning to regularize irregular building footprints from OpenStreetMap data. Poly-GAN aims to optimize polygonal representations of buildings for digital maps, addressing limitations of traditional methods in accuracy and efficiency."	"Niroshan, L. and Carswell, J.D., 2023, June. Poly-GAN: Regularizing Polygons with Generative Adversarial Networks. In International Symposium on Web and Wireless Geographical Information Systems (pp. 179-193). Cham: Springer Nature Switzerland."	https://link.springer.com/chapter/10.1007/978-3-031-34612-5_13	https://lasith-niro.github.io/files/polygan.pdf
10/06/2022	Machine Learning with Kay	AGILE: GIScience Series	"This paper explores using Ireland's national supercomputer, Kay, to train Deep Learning models for rem

## Import pandas

We are using the very handy pandas library for dataframes.

In [16]:
import pandas as pd

## Import TSV

Pandas makes this easy with the read_csv function. We are using a TSV, so we specify the separator as a tab, or `\t`.

I found it important to put this data in a tab-separated values format, because there are a lot of commas in this kind of data and comma-separated values can get messed up. However, you can modify the import statement, as pandas also has read_excel(), read_json(), and others.

In [17]:
publications = pd.read_csv("publications.tsv", sep="\t", header=0, encoding="utf-8")
publications

,pub_date,title,venue,excerpt,citation,url_slug,paper_url
0,05/06/2023,Poly-GAN: Regularizing Polygons with Generativ...,Lecture Notes in Computer Science,"This paper introduces Poly-GAN, a data-driven ...","Niroshan, L. and Carswell, J.D., 2023, June. P...",https://link.springer.com/chapter/10.1007/978-...,https://lasith-niro.github.io/files/polygan.pdf
1,10/06/2022,Machine Learning with Kay,AGILE: GIScience Series,This paper explores using Ireland's national s...,"Niroshan, L. and Carswell, J.D., 2022. Machine...",kay,https://lasith-niro.github.io/files/ml-with-ka...
2,19/05/2022,Post-analysis of OSM-GAN Spatial Change Detection,ISPRS International Journal of Geo-Information,This paper presents a post-analysis of OSM-GAN...,"Niroshan, L. and Carswell, J.D., 2022. Post-an...",post-osm-gan,https://lasith-niro.github.io/files/post-osm-g...
3,10/06/2022,OSM-GAN: Using Generative Adversarial Networks...,International Conference on Geoinformatics and...,"This paper introduces OSM-GAN, a novel approac...","Niroshan, L. and Carswell, J.D., 2022. OSM-GAN...",osm-gan,https://lasith-niro.github.io/files/osm-gan.pdf
4,07/02/2020,DeepMapper : Automatic Updating Crowdsourced Maps,TUDublin Annual Research Symposium,This poster summarizes the proposed research o...,"Hewa Manage, Lasith Niroshan and Carswell, Jam...",deepmapper,https://lasith-niro.github.io/files/deepmapper...


## Escape special characters

YAML is very picky about how it takes a valid string, so we are replacing single and double quotes (and ampersands) with their HTML encoded equivilents. This makes them look not so readable in raw format, but they are parsed and rendered nicely.

In [18]:
html_escape_table = {
    "&": "&amp;",
    '"': "&quot;",
    "'": "&apos;"
    }

def html_escape(text):
    """Produce entities within text."""
    return "".join(html_escape_table.get(c,c) for c in text)

## Creating the markdown files

This is where the heavy lifting is done. This loops through all the rows in the TSV dataframe, then starts to concatentate a big string (```md```) that contains the markdown for each type. It does the YAML metadata first, then does the description for the individual page.

In [19]:
import os
for row, item in publications.iterrows():
    print(f">> {item.title}")
    
    md_filename = str(item.pub_date) + "-" + item.url_slug + ".md"
    html_filename = str(item.pub_date) + "-" + item.url_slug
    year = item.pub_date[:4]
    
    ## YAML variables
    
    md = "---\ntitle: \""   + item.title + '"\n'
    
    md += """collection: publications"""
    
    md += """\npermalink: /publication/""" + html_filename
    
    if len(str(item.excerpt)) > 5:
        md += "\nexcerpt: '" + html_escape(item.excerpt) + "'"
    
    md += "\ndate: " + str(item.pub_date) 
    
    md += "\nvenue: '" + html_escape(item.venue) + "'"
    
    if len(str(item.paper_url)) > 5:
        md += "\npaperurl: '" + item.paper_url + "'"
    
    md += "\ncitation: '" + html_escape(item.citation) + "'"
    
    md += "\n---"
    
    ## Markdown description for individual page
        
    if len(str(item.excerpt)) > 5:
        md += "\n" + html_escape(item.excerpt) + "\n"
    
    if len(str(item.paper_url)) > 5:
        md += "\n[Download paper here](" + item.paper_url + ")\n" 
        
    md += "\nRecommended citation: " + item.citation
    
    md_filename = os.path.basename(md_filename)
       
    with open("../_publications/" + md_filename, 'w') as f:
        f.write(md)

>> Poly-GAN: Regularizing Polygons with Generative Adversarial Networks
>> Machine Learning with Kay
>> Post-analysis of OSM-GAN Spatial Change Detection
>> OSM-GAN: Using Generative Adversarial Networks for Detecting Change in High-Resolution Spatial Images
>> DeepMapper : Automatic Updating Crowdsourced Maps


These files are in the publications directory, one directory below where we're working from.

In [20]:
!ls ../_publications/

2020-deepmapper.md      2022-osm-gan.md         978-3-031-34612-5_13.md
2022-kay.md             2022-post-osm-gan.md


In [21]:
!cat ../_publications/978-3-031-34612-5_13.md

---
title: "Poly-GAN: Regularizing Polygons with Generative Adversarial Networks"
collection: publications
permalink: /publication/05/06/2023-https://link.springer.com/chapter/10.1007/978-3-031-34612-5_13
excerpt: 'This paper introduces Poly-GAN, a data-driven approach utilizing Deep Learning to regularize irregular building footprints from OpenStreetMap data. Poly-GAN aims to optimize polygonal representations of buildings for digital maps, addressing limitations of traditional methods in accuracy and efficiency.'
date: 05/06/2023
venue: 'Lecture Notes in Computer Science'
paperurl: 'https://lasith-niro.github.io/files/polygan.pdf'
citation: 'Niroshan, L. and Carswell, J.D., 2023, June. Poly-GAN: Regularizing Polygons with Generative Adversarial Networks. In International Symposium on Web and Wireless Geographical Information Systems (pp. 179-193). Cham: Springer Nature Switzerland.'
---
This paper introduces Poly-GAN, a data-driven approach utilizing Deep Learning to regularize irreg